In [1]:
## LOAD MODULES ##
import json
import librosa
import numpy as np
import os
from skimage.feature import peak_local_max
import torch
from tqdm import tqdm
import model.utils as utils
import model.net as net
from collections import OrderedDict
import model.data_loader as data_loader
import pandas as pd
import torchvision

C:\Users\vn71838\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vn71838\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\vn71838\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vn71838\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
model_dir = 'experiments/trained_model'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('My device is using a', device)

My device is using a cpu


In [4]:
## CODE TO RE-RUN TRAINED MODEL OVER THE NEW DATA ##
def batch_to_device(batch, device):
    batch_dict = {key: batch[key].to(device) for key in batch}
    return batch_dict

def unravel_index(indices, shape):
    r"""Converts a tensor of flat indices into a tensor of coordinate vectors.
    This is a `torch` implementation of `numpy.unravel_index`.
    Args:
        indices: A tensor of flat indices, (*,).
        shape: The target shape.
    Returns:
        The unraveled coordinates, (*, D).
    """
    coords = []
    for dim in reversed(shape):
        coords.append(indices % dim)
        indices = torch.div(indices, dim, rounding_mode='floor')
    coords = torch.stack(coords[::-1], dim=-1)
    return coords

def detect_objects(M, width=21, score_threshold=0.0, iou_threshold=0.45, cuda=False):
    scores = M.flatten(start_dim=0)
    boxes = unravel_index(torch.arange(M.numel()), M.shape)
    boxes = torch.cat((boxes, width * torch.ones_like(boxes)), 1).float() # 'cxcywh'
    if cuda:
        boxes = boxes.cuda()
    boxes = torchvision.ops.box_convert(boxes, in_fmt='cxcywh', out_fmt='xyxy')
    boxes = torchvision.ops.clip_boxes_to_image(boxes, M.shape[::-1])
    keep = torchvision.ops.remove_small_boxes(boxes, min_size=width)
    boxes = boxes[keep]
    scores = scores[keep]
    keep = scores.ge(score_threshold)
    boxes = boxes[keep,:]
    scores = scores[keep]
    keep = torchvision.ops.nms(boxes, scores, iou_threshold=iou_threshold)
    boxes = boxes[keep]
    scores = scores[keep]
    return boxes

@torch.no_grad()
def predict(data_dir, model_dir, clip_duration=20, limit=None, score_threshold=0.5, iou_threshold=0.05, reverse_sort=True):
    json_path = os.path.join(model_dir, 'params.json')
    assert os.path.isfile(
        json_path), "No json configuration file found at {}".format(json_path)
    params = utils.Params(json_path)
    params.cuda = torch.cuda.is_available()
    print('GPU: {}'.format(params.cuda))
    params.batch_size = 1
    # compute frequencies tuned to the mel scale
    bin_freqs = librosa.mel_frequencies(n_mels=params.n_mels, fmin=params.fmin, fmax=params.fmax, htk=True)
    model = net.Net(params).cuda() if params.cuda else net.Net(params)
    # Reload weights from the saved file
    _ = utils.load_checkpoint(os.path.join(model_dir, 'best.pth.tar'), model)    
    # put model in eval mode
    model.eval()
    
    ########################################
    # normalisation to approx Raven Pro v1.5
    ########################################
    db_offset = 0
    
    # calculate float sample normalisation
    pcm16_gain = int("0x7FFF", 0)
    db_offset += librosa.power_to_db(pcm16_gain**2)
    
    # calculate STFT normalisation
    n_fft = params.window_size
    win = np.hanning(n_fft)
    stft_normfact = n_fft / (0.5 * n_fft * (win*win).sum())
    db_offset += librosa.power_to_db(stft_normfact)
    
    # calculate mel filterbank normalisation
    mel_basis = model.mel_spec.mel_scale.fb.cpu().numpy().T
    mel_basis_normfact = np.reciprocal(np.clip(np.sum(mel_basis, axis=1, keepdims=True), a_min=1e-10, a_max=None))
    ########################################
    
    files = os.listdir(data_dir)    
    ext = [".wav", ".WAV"]    
    files = [os.path.join(data_dir, f) for f in files if f.endswith(tuple(ext))]
    files.sort(reverse=reverse_sort)
    if limit is not None:
        files = files[:limit]

    for path in tqdm(files):
        df = []

        duration = librosa.get_duration(filename=path) # duration in seconds
        num_clips = max(int(duration//clip_duration), 1)
        offset = 0

        for idx in range(num_clips):
            x, sr = librosa.load(path, sr=params.sample_rate, mono=False, offset=offset, duration=clip_duration)

            if (x.ndim == 2):
                x = x[0,:] # left channel

            # remove DC
            x -= np.mean(x)
        
            batch = {
                "input": torch.tensor(x).unsqueeze(0),
                "target": torch.tensor(0).unsqueeze(0),
                "mask": torch.tensor(0).unsqueeze(0),
            }
        
            if params.cuda:
                batch = batch_to_device(batch, 'cuda')

            output_dict = model(batch)
            mel_spec = output_dict['mel_power_spec'][0,0,:,:].cpu().numpy()
            mel_spec *= mel_basis_normfact

            scores = output_dict['logits_raw'][0,0,:,:].sigmoid()
            boxes = detect_objects(scores, width=params.box_width, score_threshold=score_threshold, iou_threshold=iou_threshold, cuda=params.cuda)
            boxes = boxes.long().cpu().numpy()
            
            n_detections = boxes.shape[0]
            for i in range(n_detections):
                x1, y1, x2, y2 = boxes[i]
                mel_bin = int((x1+x2)//2)
                frame = int((y1+y2)//2)
                score = scores[x1:x2,y1:y2].max().item()
                peak_power = mel_spec[x1:x2,y1:y2].max()
                peak_power = librosa.power_to_db(peak_power) + db_offset
                average_power = mel_spec[x1:x2,y1:y2].mean()
                average_power = librosa.power_to_db(average_power) + db_offset
                time = librosa.frames_to_time(frame, sr=sr, hop_length=params.hop_length) + offset
                
                d = OrderedDict()
                d['File'] = path
                d['Time (s)'] = time
                d['Freq (Hz)'] = bin_freqs[mel_bin]
                d['Peak Power (dB)'] = peak_power
                d['Avg Power (dB)'] = average_power
                d['Score'] = score
                df.append(d)

            # advance offset: non-overlapping clips
            offset += clip_duration

        # creates one csv file per site
        if df:
            columns = list(df[0].keys())
            df = pd.DataFrame(df, columns=columns)
            site = os.path.basename(path).split('_')[0]
            csv_path = os.path.join(data_dir, site + '.csv')
            header = not os.path.exists(csv_path)
            df.to_csv(csv_path, mode='a', index=False, header=header)

model_dir = 'experiments/trained_model'
data_dir = 'data/newfiles'
predict(data_dir, model_dir, clip_duration=20, limit=None, score_threshold=0.2, iou_threshold=0.05, reverse_sort=False)

GPU: False


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [17:27<00:00, 209.47s/it]
